In [151]:
import numpy as np
from scipy.stats import t
from numpy.linalg import inv
import random

np.set_printoptions(suppress=True)

In [152]:
def z_normalize(X, X_mean=None, X_std=None):
    """
    Z-normalize the independent variables

    Given:
    -------------------
    X : (k x n) array
        The independent variables to z-normalize
    X_mean : (k x 1) array | None
        The means of the independent variables.  If None, they are estimated
    X_std : (k x 1) array | None
        The standard deviations of the independent variables.  If None, they are estimated
        
    Produce:
    -------------------
    Z_1 : (k+1 x n) array
        The z-normalized independent variables, with a 1's vector appended
    Z : (k x n) array
        The z-normalized independent variables, without a 1's vector appended
    X_mean: (k x 1) array
        The means of the independent variables
    X_std: (k x 1) array
        The standard deviations of the independent variables.
    """
    n = X.shape[1]
    
    if X_mean is None:
        X_mean = np.mean(X)
    if X_std is None:
        X_std = np.std(X)
    
    Z = (X - X_mean)/X_std
    Z_1 = np.vstack((Z, np.ones((1,n))))
    
    return Z_1, Z, X_mean, X_std

In [153]:
# Generate synthetic data
#  - B: The actual model parameters; should be a 1xk array
#  - n: The number of synthetic subjects to generate
#  - k: The number of independent variables
#  - S_YdotX: The standard deviation of the residual
#  - X_range: The range of values for the ind vars, should be a 2-length tuple
#  - X_are_ints: When generating ind vars for the subject, should they be ints or floats?

def gen_synth_norm_data(B,n=100,S_YdotX=None,X_range=(0,10),X_are_ints=True):
    '''
    Generate synthetic normative data

    Given:
    -------------------
    B : (m x k+1) array
        The actual model parameters
    n : int
        The number of synthetic subjects to generate
    S_YdotX : m-length 1d array
        The standard deviation of the residual
    X_range: 2-length tuple
        The range of values for the ind vars
    X_are_int: Boolean
        If True, generated independent variables are Ints
        If False, generated independent variables are Floats

    Produce:
    -------------------
    X : A (k x n) array
        The sythetically generated array of independent variables
    Y : A (m x n) array
        The dependent variables
    epsilon : A (m x n) array
        The residuals
    '''

    # The number of dep vars
    m = B.shape[0]
    # The number of ind vars
    k = B.shape[1] - 1
    
    if X_are_ints:
        X = np.random.randint(X_range[0], X_range[1], size=(k, n))
    else:
        X = np.random.uniform(X_range[0], X_range[1], size=(k, n))

    if S_YdotX is None:
        S_YdotX = np.tile(1.0, (m))
    assert(S_YdotX.shape == (m,)) 
    
    Z_1, _, _, _ = z_normalize(X)

    scale_array = np.tile(S_YdotX_actual, (n,1)).T
    epsilon = np.random.normal(loc=0, scale=scale_array, size=(m,n))
    Y = B @ Z_1 + epsilon

    return X, Y, epsilon

In [154]:
def estimate_model_params(X,Y):
    '''
    Estimate model parameters from a normative set of independent and dependent variables

    Given:
    -------------------
    X : A (k x n) array
        The independent variables
    Y : A (m x n) array
        The dependent variables
        
    Produce:
    -------------------
    B_estimate: (m x k+1) array
        The estimated model parameters
    S_YdotX_estimate: (m,) array
        The estimated standard deviation of the residual
    R : (k x k) array
        The inverse of the correlation matrix (R = (ZZ^T)^-1)
    X_mean: (k x 1) array
        The means of the independent variables
    X_std: (k x 1) array
        The standard deviations of the independent variables.
    '''
    Z_1, Z, X_mean, X_std = z_normalize(X)
    R = inv(Z @ Z.T)
    B_estimate = Y @ Z_1.T @ inv(Z_1 @ Z_1.T)
    
    Y_estimate = B_estimate @ Z_1
    S_YdotX_estimate = np.std(Y - Y_estimate, axis=1)

    return B_estimate, S_YdotX_estimate, R, X_mean, X_std

In [155]:
def gen_synth_sub_data(B, n, epsilon, X_range, X_mean, X_std, X_are_ints=True):
    '''
    Generate synthetic subject data, to be evaluated using model pramaters from normative data

    Given:
    -------------------
    B: (m x k+1) array
        The model parameters used to generate the independent vars
    n : int
        The number of synthetic subjects to generate
    epsilon: float | (k x n) array
        The residuals to use when generating data.  If epsilon is a single float, the same
        resdual is used for all dependend variables and all subjects
    X_range: 2-length tuple
        The range of values for the ind vars
    X_mean: (k x 1) array
        The means of the independent variables from the normative set
    X_std: (k x 1) array
        The standard deviations of the independent variables from the normative set    
    X_are_int: Boolean
        If True, generated independent variables are Ints
        If False, generated independent variables are Floats
        
    Produce:
    -------------------
    X : A (k x n) array
        The sythetically generated array of independent variables
    Y : A (m x n) array
        The dependent variables
    '''
    # The number of dep vars
    m = B.shape[0]
    # The number of ind vars
    k = B.shape[1] - 1

    if isinstance(epsilon, float):
        epsilon = np.tile(epsilon, (m,n))
    assert(epsilon.shape == (m,n))

    if X_are_ints:
        X = np.random.randint(X_range[0], X_range[1], size=(k, n))
    else:
        X = np.random.uniform(X_range[0], X_range[1], size=(k, n))

    Z_1, _, _, _= z_normalize(X, X_mean, X_std)
    Y = B @ Z_1 + epsilon
    
    return X, Y

In [198]:
# Parameters for normative synthetic data generation

# number of samples
n = 1000
# number of ind vars
k = 3
# number of dep vars
m = 2
# std_dev of noise
#S_YdotX_actual = np.random.uniform(1.0, 2.0, size=(m))
S_YdotX_actual = np.random.uniform(0.001, 0.001, size=(m))

# B (model params) will be chosen randomly from this range
B_range = (0, 21)
B_actual = np.random.randint(B_range[0], B_range[1], size=(m, k+1))
#B = np.random.uniform(B_range[0], B_range[1], size=(m, k+1))
        
# X (ind vars) will be chosen randomly from this range
X_range = (1,6)
X_are_ints = True

In [199]:
# Generate synthetic normative dataset
X, Y, epsilon_actual = gen_synth_norm_data(B_actual, n, S_YdotX_actual, X_range, X_are_ints=X_are_ints)

# estimate model params from synthetic dataset
B_estimate, S_YdotX_estimate, R, X_mean, X_std = estimate_model_params(X,Y)

In [200]:
print('B_actual:')
print(B_actual)
print()
print('B_estimate')
print(B_estimate)
print()
print('S_YdotX_actual')
print(S_YdotX_actual)
print()
print('S_YdotX_estimate')
print(S_YdotX_estimate)

B_actual:
[[ 7  7 19 15]
 [ 4  8 12  2]]

B_estimate
[[ 7.00000358  6.99999972 19.00000897 15.00000529]
 [ 3.99998478  8.00002978 11.99998654  2.00000366]]

S_YdotX_actual
[0.001 0.001]

S_YdotX_estimate
[0.0009866  0.00103336]


In [201]:
# Generate a single perfectly median subject
x_obs, y_obs = gen_synth_sub_data(B_actual, 1, 0.0, X_range, X_mean, X_std, X_are_ints=X_are_ints)

In [202]:
print(x_obs)
print(y_obs)
print(S_YdotX_estimate)

[[5]
 [2]
 [4]]
[[33.61547693]
 [10.60220082]]
[0.0009866  0.00103336]


In [207]:
def single_subject_eval(x_obs, y_obs, B, R, S_YdotX, X_mean, X_std):
    # z-normalize the subjects
    z_obs1, z_obs, _, _ = z_normalize(x_obs, X_mean, X_std)
    y_estimate = B @ z_obs1
    
    r_A = np.sum(np.diag(R) * (z_obs.T ** 2))
    
    # Computing r_B:
    # ------------------
    # the upper matrix elements, excluding the diagonal
    #  - https://stackoverflow.com/a/47314816
    # - we pass `k=1` to triu_indices() to exclude diagonal elements
    uR_idx = np.triu_indices(R.shape[0], k=1)

    # This is the r_{i,j} term of the equation for r_B
    r_i_j = R[uR_idx[0],uR_idx[1]]
    # This is the z_{obs,i} term of the equation for r_B
    z_obs_i = z_obs[uR_idx[0]].T
    # This is the z_{obs,j} term for the equation for r_B
    z_obs_j = z_obs[uR_idx[1]].T

    r_B = np.sum(r_i_j * z_obs_i * z_obs_j)

    # Compute S_{N+1}
    S_Nplus1 = S_YdotX * np.sqrt(1 + 1/n + 1/(n-1)*r_A + 2/(n-1)*r_B)
    
    # Compute the t-statistic
    t_diff = (y_obs - y_estimate).T/S_Nplus1
    
    # Compute the percentile estimate, since this is a perfectly median subject,
    # It should be ~0.5.  The larger n, the closer this should be to 0.5
    p = t.cdf(x=t_diff, df=n-k)
    print(f'p: {p}')

In [208]:
single_subject_eval(x_obs, y_obs, B_estimate, R, S_YdotX_estimate, X_mean, X_std)



p: [[0.49311925 0.51888627]]


# -------------------------------------------------------

In [11]:

    
#def gen_synth_sub_data(B, n, epsilon, X_range, X_mean, X_std, X_are_ints=True):
    

# def gen_synth_sub_data(B, n, k, epsilon, X_range, X_mean, X_std, X_are_ints=True):
#gen_synth_sub_data(B_actual, 1, k, [0.0], X_range, X_mean, X_std, X_are_ints=True)

# very basic test data

# number of samples
n = 10
# number of ind vars
k = 3
# number of dep vars
m = 1
# std_dev of noise
S_YdotX_actual = 1.0
# actual model params
B_actual_min_range = 0
B_actual_max_range = 21
B_actual = np.random.randint(B_actual_min_range, B_actual_max_range , size=(m, k+1))

# Z-normalize ind vars (X) and append a ones column for linear offests (Z_1)
X_min_range = 1
X_max_range = 6
X = np.random.randint(X_min_range, X_max_range, size=(k, n))
X_mean = np.mean(X)
X_std = np.std(X)
Z = (X - X_mean)/X_std
Z_1 = np.vstack((Z, np.ones((1,n))))

# Generate dep vars
epsilon_actual = np.random.normal(loc=0, scale=S_YdotX_actual, size=(1,n))
Y = B_actual @ Z_1 + epsilon_actual

In [33]:
#---- 
# Estimate model parameters
B_estimate = Y @ Z_1.T @ inv(Z_1 @ Z_1.T)
R = inv(Z @ Z.T)
Y_estimate = B_estimate @ Z_1
S_YdotX_estimate = np.std(Y - Y_estimate)

print('B_actual:')
print(f'{B_actual}')
print()
print('B_estimate:')
print(f'{B_estimate}')
print()
print(f'S_YdotX_actual:   {S_YdotX_actual}')
print(f'S_YdotX_estimate: {S_YdotX_estimate}')

B_actual:
[[ 4 20  1 17]]

B_estimate:
[[ 3.48204262 20.22701472  1.40512833 16.89608109]]

S_YdotX_actual:   1.0
S_YdotX_estimate: 1.0659957890883622


In [34]:
# ----
# Generate a perfectly median subject:
x_obs = np.random.randint(X_min_range, X_max_range, size=(k, 1))
z_obs = (x_obs - X_mean)/X_std
z_obs1 = np.vstack((z_obs, 1))

# Compute the subjects 'actual' (observed) dep vars and estimated dep vars
y_obs = B_actual @ z_obs1
y_estimate = B_estimate @ z_obs1

print()
print(f'y_obs.T:          {y_obs.T}')
print(f'y_estimate.T:     {y_estimate.T}')


y_obs.T:          [[5.40020735]]
y_estimate.T:     [[3.83077436]]


In [31]:
r_A = np.sum(np.diag(R) * (z_obs.T ** 2))

# Computing r_B:
# ------------------
# the upper matrix elements, excluding the diagonal
#  - https://stackoverflow.com/a/47314816
# - we pass `k=1` to triu_indices() to exclude diagonal elements
uR_idx = np.triu_indices(R.shape[0], k=1)

# This is the r_{i,j} term of the equation for r_B
r_i_j = R[uR_idx[0],uR_idx[1]]
# This is the z_{obs,i} term of the equation for r_B
z_obs_i = z_obs[uR_idx[0]].T
# This is the z_{obs,j} term for the equation for r_B
z_obs_j = z_obs[uR_idx[1]].T

r_B = np.sum(r_i_j * z_obs_i * z_obs_j)

# Compute S_{N+1}
S_Nplus1 = S_YdotX_estimate * np.sqrt(1 + 1/n + 1/(n-1)*r_A + 2/(n-1)*r_B)

# Compute the t-statistic
t_diff = (y_obs - y_estimate)/S_Nplus1

# Compute the percentile estimate, since this is a perfectly median subject,
# It should be ~0.5.  The larger n, the closer this should be to 0.5
p = t.cdf(x=t_diff, df=n-k)
print(f'p: {p}')

NameError: name 'z_obs' is not defined

In [60]:
print(R)
print(x_obs)
print(z_obs)
print(r_i_j)
print(z_obs_i)
print(z_obs_j)
print(uR_idx[0])
print(uR_idx[1])

[[ 0.01018605 -0.00197682 -0.00023317]
 [-0.00197682  0.01040471  0.00140284]
 [-0.00023317  0.00140284  0.0105513 ]]
[[2]
 [3]
 [2]]
[[-0.69340571]
 [-0.01136731]
 [-0.69340571]]
[-0.00197682 -0.00023317  0.00140284]
[[-0.69340571 -0.69340571 -0.01136731]]
[[-0.01136731 -0.69340571 -0.69340571]]
[0 0 1]
[1 2 2]


In [29]:
r_i_j

array([1.64313830e-05, 3.63475799e-05, 2.49454122e-06])

In [5]:
off_d_R_idx = np.where(~np.eye(R.shape[0],dtype=bool))

In [6]:
~np.eye(R.shape[0],dtype=bool)

array([[False,  True,  True,  True,  True],
       [ True, False,  True,  True,  True],
       [ True,  True, False,  True,  True],
       [ True,  True,  True, False,  True],
       [ True,  True,  True,  True, False]])

In [10]:
R_test = R.copy()

(array([0, 0, 0, 0, 1, 1, 1, 2, 2, 3]), array([1, 2, 3, 4, 2, 3, 4, 3, 4, 4]))

In [9]:
off_d_R_idx 

(array([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4]),
 array([1, 2, 3, 4, 0, 2, 3, 4, 0, 1, 3, 4, 0, 1, 2, 4, 0, 1, 2, 3]))